In [1]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import math
import h5py
import warnings
from tensorflow.python.framework import ops
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
h2o.init(strict_version_check=False) # start h2o

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_191"; Java(TM) SE Runtime Environment (build 1.8.0_191-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from /Users/bonnie/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/f1/2lb7b2n57j9d7ktf62rd4mkh0000gn/T/tmp3258zbck
  JVM stdout: /var/folders/f1/2lb7b2n57j9d7ktf62rd4mkh0000gn/T/tmp3258zbck/h2o_bonnie_started_from_python.out
  JVM stderr: /var/folders/f1/2lb7b2n57j9d7ktf62rd4mkh0000gn/T/tmp3258zbck/h2o_bonnie_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,13 days
H2O cluster name:,H2O_from_python_bonnie_w9egm5
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [3]:
train = pd.read_csv('data/trainPrice.csv')
test = pd.read_csv('data/testPrice.csv')
train.describe()

,key,apartment_id,city,transaction_year_month,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,...,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price
count,1.601458e+06,1.601458e+06,1.601458e+06,1.601458e+06,1.601458e+06,1.601458e+06,1.601458e+06,1.601458e+06,1.601458e+06,1.601458e+06,...,1.601458e+06,1.601458e+06,1.601449e+06,1.601449e+06,1.601458e+06,1.601458e+06,1.601458e+06,1.600767e+06,1.600767e+06,1.601458e+06
mean,8.007597e+05,7.781149e+03,6.252877e-01,2.012416e+05,1.998645e+03,7.946863e+01,1.013031e+01,3.666195e+01,1.277745e+02,1.704232e+09,...,1.181628e+03,1.274730e+01,2.077357e+01,1.393617e+01,3.481310e+04,1.020035e+02,2.931756e+02,2.951266e+00,1.587173e+00,4.021752e+08
std,4.623505e+05,7.384526e+03,4.840487e-01,3.775998e+02,8.444635e+00,2.800054e+01,7.194129e+00,1.157632e+00,1.005557e+00,7.216393e+08,...,1.128092e+03,1.399157e+01,8.750388e+00,6.785973e+00,4.157775e+04,3.306049e+01,3.237872e+02,6.653449e-01,5.071750e-01,3.223861e+08
min,0.000000e+00,3.000000e+00,0.000000e+00,2.006010e+05,1.962000e+03,1.639600e+01,-4.000000e+00,3.504879e+01,1.268061e+02,1.111012e+09,...,9.900000e+01,1.000000e+00,5.000000e+00,1.000000e+00,1.300000e+01,2.010000e+01,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+06
25%,4.003642e+05,1.606000e+03,0.000000e+00,2.009070e+05,1.993000e+03,5.984000e+01,5.000000e+00,3.518907e+01,1.269782e+02,1.135011e+09,...,4.160000e+02,4.000000e+00,1.500000e+01,9.000000e+00,6.610000e+03,7.993000e+01,9.000000e+01,3.000000e+00,1.000000e+00,1.990000e+08
50%,8.007305e+05,5.579000e+03,1.000000e+00,2.013080e+05,1.999000e+03,8.446000e+01,9.000000e+00,3.749926e+01,1.270743e+02,1.165011e+09,...,8.110000e+02,9.000000e+00,2.000000e+01,1.300000e+01,2.048400e+04,1.029600e+02,1.940000e+02,3.000000e+00,2.000000e+00,3.240000e+08
75%,1.201101e+06,1.106300e+04,1.000000e+00,2.016030e+05,2.005000e+03,8.498000e+01,1.400000e+01,3.756554e+01,1.290298e+02,2.632010e+09,...,1.590000e+03,1.600000e+01,2.500000e+01,1.700000e+01,4.221900e+04,1.115400e+02,3.840000e+02,3.000000e+00,2.000000e+00,5.000000e+08
max,1.605373e+06,5.026400e+04,1.000000e+00,2.018100e+05,2.018000e+03,2.954340e+02,8.000000e+01,3.768763e+01,1.292376e+02,2.671033e+09,...,6.864000e+03,1.240000e+02,8.000000e+01,7.000000e+01,1.922520e+05,4.234000e+02,2.960000e+03,8.000000e+00,5.000000e+00,8.200000e+09


In [4]:
test.describe()

,key,apartment_id,city,transaction_year_month,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,...,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price
count,3.918000e+03,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3.918000e+03,...,3918.000000,3918.000000,3917.000000,3917.000000,3918.000000,3918.00000,3918.000000,3913.000000,3913.000000,3918.0
mean,1.590645e+06,9749.125064,0.673047,201806.033180,1998.906075,82.191308,9.149056,36.771194,127.668297,1.633625e+09,...,510.825676,6.040837,18.296145,12.827164,43443.242726,104.43292,155.846350,3.009200,1.641963,0.0
std,4.151869e+04,9086.376371,0.469160,26.135513,9.715528,28.127419,6.679446,1.119282,0.977821,6.992680e+08,...,575.971467,7.166845,7.497573,6.705995,45197.995354,32.98825,182.491791,0.647804,0.501885,0.0
min,4.625330e+05,3.000000,0.000000,200912.000000,1962.000000,16.536000,-1.000000,35.048786,126.746463,1.111012e+09,...,99.000000,1.000000,5.000000,1.000000,85.000000,25.12000,0.000000,0.000000,0.000000,0.0
25%,1.590730e+06,2943.250000,0.000000,201808.000000,1993.000000,59.940000,4.000000,35.202301,126.948060,1.138011e+09,...,180.000000,2.000000,15.000000,8.000000,10028.750000,81.79000,48.000000,3.000000,1.000000,0.0
50%,1.600271e+06,6341.000000,1.000000,201809.000000,1999.000000,84.600000,8.000000,37.506656,127.059260,1.159011e+09,...,305.000000,4.000000,17.000000,12.000000,31356.000000,103.92000,98.000000,3.000000,2.000000,0.0
75%,1.603609e+06,15053.000000,1.000000,201809.000000,2005.000000,84.980000,13.000000,37.560671,129.013425,2.626011e+09,...,603.000000,8.000000,23.000000,16.000000,48543.500000,112.42500,193.000000,3.000000,2.000000,0.0
max,1.605375e+06,50264.000000,1.000000,201810.000000,2018.000000,257.260000,57.000000,37.687633,129.237592,2.824511e+09,...,6864.000000,124.000000,80.000000,70.000000,168630.000000,324.01000,2960.000000,6.000000,4.000000,0.0


In [5]:
# conversion of pandas dataframe to h2o frame
hf = h2o.H2OFrame(train)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
hf.describe()

Rows:1601458
Cols:25




,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
type,int,int,int,int,enum,int,real,int,real,real,int,int,int,int,int,int,enum,enum,int,real,int,int,int,enum,int
mins,0.0,3.0,0.0,200601.0,,1962.0,16.396,-4.0,35.04878639164053,126.80610153120372,1111011500.0,0.0,99.0,1.0,5.0,1.0,,,13.0,20.1,0.0,0.0,0.0,,7000000.0
mean,800759.7448175347,7781.149295829175,0.6252877065773813,201241.55087676362,,1998.6448149124114,79.46862932911755,10.130310005007935,36.661949473346986,127.77449044480244,1704231534.4890609,1350.1904348373296,1181.6275163007713,12.747295901609656,20.773567562875865,13.93617030576684,,,34813.10043660217,102.0035307638414,293.1755818760154,2.951266486627974,1.5871728989915466,,402175230.25892633
maxs,1605373.0,50264.0,1.0,201810.0,,2018.0,295.434,80.0,37.68763299875252,129.237592,2671033029.0,9766.0,6864.0,124.0,80.0,70.0,,,192252.0,423.4,2960.0,8.0,5.0,,8200000000.0
sigma,462350.5061373109,7384.526451084999,0.4840486926815515,377.5998216160958,,8.444635366614945,28.000543057488933,7.194128872604277,1.157632092872639,1.0055565155109043,721639314.9042882,1494.4727041151286,1128.0924817332586,13.991570692148471,8.750387946359558,6.7859730213432705,,,41577.75287061174,33.060492260811195,323.7871566888945,0.6653449077649864,0.5071749594056251,,322386136.9694438
zeros,1,0,600086,0,,0,0,0,0,0,0,823,0,0,0,0,,,0,0,16456,3231,3231,,0
missing,0,0,0,0,0,0,0,0,0,0,0,91813,0,0,9,9,0,0,0,0,0,691,691,0,0
0,0.0,5584.0,1.0,200601.0,11~20,1999.0,47.43,6.0,37.58596505640614,127.00023096787316,1111017100.0,163.0,136.0,1.0,8.0,4.0,individual,gas,91120.0,65.63,46.0,1.0,1.0,corridor,215000000.0
1,1.0,5584.0,1.0,200601.0,11~20,1999.0,44.37,8.0,37.58596505640614,127.00023096787316,1111017100.0,163.0,136.0,1.0,8.0,4.0,individual,gas,91119.0,61.39,10.0,2.0,1.0,corridor,200000000.0
2,2.0,5059.0,1.0,200601.0,11~20,1992.0,54.7,8.0,37.58051141258221,127.0140155572653,1111017400.0,902.0,585.0,5.0,14.0,9.0,individual,gas,8430.0,72.36,201.0,2.0,1.0,corridor,168000000.0


In [12]:
target = 'transaction_real_price'

In [13]:
y=target
X=[name for name in hf.columns if name != y]
print(X)
print('target :',y)

['key', 'apartment_id', 'city', 'transaction_year_month', 'transaction_date', 'year_of_completion', 'exclusive_use_area', 'floor', 'latitude', 'longitude', 'address_by_law', 'total_parking_capacity_in_site', 'total_household_count_in_sites', 'apartment_building_count_in_sites', 'tallest_building_in_sites', 'lowest_building_in_sites', 'heat_type', 'heat_fuel', 'room_id', 'supply_area', 'total_household_count_of_area_type', 'room_count', 'bathroom_count', 'front_door_structure']
target : transaction_real_price


In [14]:
# Set up AutoML
aml = H2OAutoML(max_runtime_secs=300 ,exclude_algos = ['DeepLearning'])

In [17]:
aml.train(x=X,y=y,training_frame=hf)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [18]:
print(aml.leaderboard)


This H2OFrame is empty.

